In [ ]:
import torch

torch.cuda.get_device_name(0)
print(torch.__version__)

torch.cuda.is_available()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.maual_seed(123)

if devive == 'cuda':
  torch.cuda.manual_seed_all(123)

In [ ]:
''' VGG 11,113,16,19 in Pytorch '''

import torch
import torch.nn as nn

cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
  #VGG 클래스 불러올시, vgg_name 안에 입력하기 -> 바로 make_layers(cfg[vgg_name]) 함수 사용
  def __init__(self, vgg_name):
    super(VGG, self).__init__()
    self.features = self._make_layers(cfg[vgg_name])

    #마지막에 FC 층을 의미
    self.classifier = nn.Linear(512,10)

    #forward propagation
    def forward(self, x):
      out = self.features(x) # out에 features(x)를 받는다
      out = out.view(out.size(0), -1)
      out = self.classifier(out) #classifier 마지막 FC 층 의미(?)

    #층 만들기
    def _make_layers(self, cfg):
      layers =[] #빈 층 리스트
      in_channels = 3


      #cfg dictionary 를 기반으로
      for x in cfg:

        #M cfg 안에 있다면, Maxpooling 층을 층 리스트에 넣음
        if x == 'M':
          layers += [nn.MaxPool2d(kernel_size =2, stride =2)]

        #M 외의 것은 convolutional 층 / 숫자는 BatchNorm2d에 입력 (?)
        else:
          layers += [nn.Conv2d(in_chnnels, x, kernel_size =3, padding=1),
                     nn.BatchNorm2d(x),
                     nn.ReLU(inplace = True)
                     ]
          in_channels = x

      # 마지막으로 AvgPool 층 추가
      layers += [nn.AvgPool2d(kernel_size =1, stride =1)]

      # Sequenetial 은 신경망 구축하는 함수임!
      return nn.Sequential(*layers)

def test():
  net = VGG('VGG16')
  x = torch.randn(2,3,32,32)
  y = net(x)
  print(y.size())

In [ ]:
'''Train CIFAR10 with PyTorch.'''

import requests
from bs4 import BeautifulSoup as bs
body = {"terminal": "1,16011301,," , "rows": 1,"visMode": 1}
r = requests.post('http://st.atb.no/New/minskjerm/DataHandler.ashx?type=departureTimes&lang=no', data = body)
soup = bs(r.content, 'lxml')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os

print('==> Preparing Data')

#transforms.Compose는 데이터 전처리 과정을 의미한다.
#train 데이터는 Crop 과 Flip을 통해 훈련할 때 Overfitting을 막아주고, 적은 데이터 단점을 보완
#Normalize는 train set 데이터들의 mean과 std를 입력
transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding =4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./cifar10', train=True, download=True, transform=transform_train
)

trainloader = torch.utils.data.DataLoader(
    trainset, batch_size = 128, shuffle = True, num_workers =2
)

testset = torchvision.datasets.CIFAR10(
    root='./cifar10', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
# Model

print('==> Building model...')

net = VGG('VGG16')

net = net.to(decive)

if device == 'cuda':
  net = torch.nn.DataParallel(net)
  cudnn.benchmark = True


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
            

for epoch in range(start_epoch, start_epoch+200):
    train(epoch)
    test(epoch)
    scheduler.step()